# Entity Extraction (NER) Model Training

This notebook trains a Named Entity Recognition (NER) model to extract entities from business documents:
- Invoice numbers
- Amounts
- Dates
- Email addresses
- Phone numbers
- Tax IDs
- Company names

## Approach
1. **Option A:** spaCy NER model (pre-trained + fine-tuning)
2. **Option B:** Transformers (BERT-based NER)
3. **Option C:** Regex + Rule-based (fast, no training needed)


In [1]:
# Verify dependencies are available (all packages are installed system-wide in Python 3.12.12)
import sys
print(f"Python version: {sys.version}")
print(f"Python executable: {sys.executable}")

try:
    import pandas as pd
    import numpy as np
    import transformers
    import torch
    import datasets
    print("✓ Core packages available!")
    print(f"  - pandas: {pd.__version__}")
    print(f"  - numpy: {np.__version__}")
    print(f"  - transformers: {transformers.__version__}")
    print(f"  - torch: {torch.__version__}")
    
    # Check spaCy
    try:
        import spacy
        print(f"  - spacy: {spacy.__version__}")
        try:
            nlp = spacy.load("en_core_web_sm")
            print("  - spaCy model 'en_core_web_sm': ✓ Loaded")
        except OSError:
            print("  - spaCy model 'en_core_web_sm': ⚠️ Not found (run: python3 -m spacy download en_core_web_sm)")
    except ImportError:
        print("  - spacy: ❌ Not installed (run: python3 -m pip install spacy)")
        print("    Then download model: python3 -m spacy download en_core_web_sm")
except ImportError as e:
    print(f"❌ Missing package: {e}")
    print("Run: python3 -m pip install <package_name>")


Python version: 3.12.12 (main, Oct  9 2025, 11:07:00) [Clang 17.0.0 (clang-1700.0.13.3)]
Python executable: /usr/local/opt/python@3.12/bin/python3.12


✓ Core packages available!
  - pandas: 2.3.3
  - numpy: 1.26.4
  - transformers: 4.57.3
  - torch: 2.2.2
  - spacy: ❌ Not installed (run: python3 -m pip install spacy)
    Then download model: python3 -m spacy download en_core_web_sm


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
import json
from typing import Dict, List

# Set paths
BASE_DIR = Path('../')
DATA_DIR = BASE_DIR / 'data'
MODELS_DIR = BASE_DIR / 'models' / 'entity_extractor'
MODELS_DIR.mkdir(parents=True, exist_ok=True)

print(f"Base directory: {BASE_DIR}")
print(f"Data directory: {DATA_DIR}")
print(f"Models directory: {MODELS_DIR}")


Base directory: ..
Data directory: ../data
Models directory: ../models/entity_extractor


## 1. Load and Prepare Data


In [3]:
# Create sample training data with entity annotations
sample_documents = [
    {
        'text': 'Invoice #INV-001\nDate: 2024-01-15\nAmount: $1,500.00\nEmail: contact@company.com\nPhone: 555-1234',
        'entities': {
            'invoice_number': 'INV-001',
            'date': '2024-01-15',
            'amount': '$1,500.00',
            'email': 'contact@company.com',
            'phone': '555-1234'
        }
    },
    {
        'text': 'Purchase Order PO-2024-001\nSupplier: ABC Corp\nOrder Date: 01/20/2024\nTotal: $5,000.00',
        'entities': {
            'po_number': 'PO-2024-001',
            'company_name': 'ABC Corp',
            'date': '01/20/2024',
            'amount': '$5,000.00'
        }
    },
    {
        'text': 'Receipt #RCP-789\nPayment Date: 2024-02-01\nAmount Paid: $750.25\nTax ID: 12-3456789',
        'entities': {
            'receipt_number': 'RCP-789',
            'date': '2024-02-01',
            'amount': '$750.25',
            'tax_id': '12-3456789'
        }
    }
]

df = pd.DataFrame(sample_documents)
print(f"✓ Created {len(df)} sample documents with entity annotations")
print(f"\nSample document:")
print(df.iloc[0]['text'])
print(f"\nEntities:")
print(df.iloc[0]['entities'])


✓ Created 3 sample documents with entity annotations

Sample document:
Invoice #INV-001
Date: 2024-01-15
Amount: $1,500.00
Email: contact@company.com
Phone: 555-1234

Entities:
{'invoice_number': 'INV-001', 'date': '2024-01-15', 'amount': '$1,500.00', 'email': 'contact@company.com', 'phone': '555-1234'}


## 2. Method A: Rule-based Entity Extraction (Fast, No Training)


In [4]:
def extract_entities_rule_based(text: str) -> Dict[str, List[str]]:
    """Extract entities using regex patterns"""
    entities = {
        'invoice_numbers': re.findall(r'(?:invoice|inv)[\s#:]*([A-Z0-9\-]+)', text, re.IGNORECASE),
        'po_numbers': re.findall(r'(?:po|purchase\s*order)[\s#:]*([A-Z0-9\-]+)', text, re.IGNORECASE),
        'amounts': re.findall(r'\$[\d,]+\.?\d*', text),
        'dates': re.findall(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}|\d{4}[/-]\d{1,2}[/-]\d{1,2}', text),
        'emails': re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', text),
        'phones': re.findall(r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b', text),
        'tax_ids': re.findall(r'(?:tax\s*id|ein|vat)[\s#:]*([A-Z0-9\-]+)', text, re.IGNORECASE),
    }
    return {k: v for k, v in entities.items() if v}

# Test
test_text = df.iloc[0]['text']
extracted = extract_entities_rule_based(test_text)
print(f"Text: {test_text}")
print(f"\nExtracted entities:")
for entity_type, values in extracted.items():
    print(f"  {entity_type}: {values}")

# Save rule-based extractor
import pickle
with open(MODELS_DIR / 'rule_based_extractor.pkl', 'wb') as f:
    pickle.dump(extract_entities_rule_based, f)
print(f"\n✓ Rule-based extractor saved to {MODELS_DIR}")


Text: Invoice #INV-001
Date: 2024-01-15
Amount: $1,500.00
Email: contact@company.com
Phone: 555-1234

Extracted entities:
  invoice_numbers: ['INV-001']
  amounts: ['$1,500.00']
  dates: ['2024-01-15']
  emails: ['contact@company.com']

✓ Rule-based extractor saved to ../models/entity_extractor


## 3. Method B: spaCy NER (Pre-trained Model)


In [5]:
try:
    import spacy
    nlp = spacy.load("en_core_web_sm")
    
    def extract_entities_spacy(text: str) -> Dict[str, List[str]]:
        """Extract entities using spaCy NER"""
        doc = nlp(text)
        entities = {
            'persons': [],
            'organizations': [],
            'dates': [],
            'money': [],
            'emails': [],
            'phones': []
        }
        
        for ent in doc.ents:
            if ent.label_ == 'PERSON':
                entities['persons'].append(ent.text)
            elif ent.label_ == 'ORG':
                entities['organizations'].append(ent.text)
            elif ent.label_ == 'DATE':
                entities['dates'].append(ent.text)
            elif ent.label_ == 'MONEY':
                entities['money'].append(ent.text)
        
        # Also extract emails and phones with regex
        entities['emails'] = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', text)
        entities['phones'] = re.findall(r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b', text)
        
        return {k: v for k, v in entities.items() if v}
    
    # Test
    test_text = df.iloc[0]['text']
    extracted = extract_entities_spacy(test_text)
    print(f"Text: {test_text}")
    print(f"\nExtracted entities (spaCy):")
    for entity_type, values in extracted.items():
        print(f"  {entity_type}: {values}")
    
    # Save
    with open(MODELS_DIR / 'spacy_extractor.pkl', 'wb') as f:
        pickle.dump(extract_entities_spacy, f)
    print(f"\n✓ spaCy extractor saved to {MODELS_DIR}")
    
except Exception as e:
    print(f"spaCy not available: {e}")
    print("Install with: pip install spacy && python -m spacy download en_core_web_sm")


spaCy not available: No module named 'spacy'
Install with: pip install spacy && python -m spacy download en_core_web_sm
